# Import

These requirements are necessary if you launch this notebook from SageMaker instances

In [1]:
"""!pip install mlflow
!pip install pytorch-lightning
!pip install transformers
!pip install tqdm
!pip install sagemaker

!pip install s3fs
!pip install smdebug"""

'!pip install mlflow\n!pip install pytorch-lightning\n!pip install transformers\n!pip install tqdm\n!pip install sagemaker\n\n!pip install s3fs\n!pip install smdebug'

In [31]:
import sys
sys.path.append('../../../')

import os
import sys
import logging
import argparse
from pathlib import Path
from typing import Any, Dict, Optional

In [32]:
from tqdm.auto import tqdm

import torchmetrics
from torchmetrics.functional import accuracy, f1, auroc

import sagemaker
from sagemaker import get_execution_role
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.core.decorators import auto_move_data
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import MLFlowLogger


import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import rc

import pandas as pd

Local constants, regarding the data, MLFlow server, paths, etc..: use them

In [33]:
from deep.constants import *
from deep.utils import *

In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data

You can use the data you want. We advise the `pandas` format.

In [35]:
DATA_PATH = os.path.join(
    '..', '..', '..', "data", "frameworks_data", 'data_v0.7.1'
)

"""
original_df = pd.read_csv(os.path.join(DATA_PATH, 'full_dataset.csv'))
augmented_data = pd.read_csv(os.path.join(DATA_PATH, 'generated_text.csv'))"""


tot_df = pd.read_csv(os.path.join(DATA_PATH, 'train_val_df.csv'))
test_df = pd.read_csv(os.path.join(DATA_PATH, 'test_v0.7.1.csv'))
"""full_df = pd.read_csv(os.path.join(DATA_PATH, 'full_dataset_with_translations.csv'))"""

"full_df = pd.read_csv(os.path.join(DATA_PATH, 'full_dataset_with_translations.csv'))"

In [27]:
def flatten(t):
    return [item for sublist in t for item in sublist]
from ast import literal_eval

tags_2d = set(flatten(tot_df.subpillars_2d.apply(literal_eval).tolist()))
tags_1d = set(flatten(tot_df.subpillars_1d.apply(literal_eval).tolist()))



set()

In [30]:
tags_1d

{'Casualties->Dead',
 'Casualties->Injured',
 'Casualties->Missing',
 'Context->Demography',
 'Context->Economy',
 'Context->Environment',
 'Context->Legal & Policy',
 'Context->Politics',
 'Context->Security & Stability',
 'Context->Socio Cultural',
 'Covid-19->Cases',
 'Covid-19->Contact Tracing',
 'Covid-19->Deaths',
 'Covid-19->Hospitalization & Care',
 'Covid-19->Restriction Measures',
 'Covid-19->Testing',
 'Covid-19->Vaccination',
 'Displacement->Intentions',
 'Displacement->Local Integration',
 'Displacement->Pull Factors',
 'Displacement->Push Factors',
 'Displacement->Type/Numbers/Movements',
 'Humanitarian Access->Number Of People Facing Humanitarian Access Constraints/Humanitarian Access Gaps',
 'Humanitarian Access->Physical Constraints',
 'Humanitarian Access->Population To Relief',
 'Humanitarian Access->Relief To Population',
 'Information And Communication->Communication Means And Preferences',
 'Information And Communication->Information Challenges And Barriers',
 'In

In [8]:
"""train_val_ids = pd.concat([train_df, val_df]).entry_id.unique()
train_val_df = full_df[full_df.entry_id.isin(train_val_ids)]

fr_df = train_val_df[['entry_id', 'translation_fr']]\
        .rename(columns={'translation_fr':'excerpt'}).dropna()
en_df = train_val_df[['entry_id', 'translation_en']]\
        .rename(columns={'translation_en':'excerpt'}).dropna()
es_df = train_val_df[['entry_id', 'translation_es']]\
        .rename(columns={'translation_es':'excerpt'}).dropna()

augmented_data = pd.concat([en_df, fr_df, es_df])


print('fr:', fr_df.shape[0], 'en:', en_df.shape[0], 'es:', es_df.shape[0])"""

"train_val_ids = pd.concat([train_df, val_df]).entry_id.unique()\ntrain_val_df = full_df[full_df.entry_id.isin(train_val_ids)]\n\nfr_df = train_val_df[['entry_id', 'translation_fr']]        .rename(columns={'translation_fr':'excerpt'}).dropna()\nen_df = train_val_df[['entry_id', 'translation_en']]        .rename(columns={'translation_en':'excerpt'}).dropna()\nes_df = train_val_df[['entry_id', 'translation_es']]        .rename(columns={'translation_es':'excerpt'}).dropna()\n\naugmented_data = pd.concat([en_df, fr_df, es_df])\n\n\nprint('fr:', fr_df.shape[0], 'en:', en_df.shape[0], 'es:', es_df.shape[0])"

In [9]:
"""def preprocess_columns(tmp_col):
    if str(tmp_col) == 'nan':
        return []
    evaluated_column = literal_eval(tmp_col)
    cleaned_column = [item for item in evaluated_column if str(item)!='NOT_MAPPED' and str(item)!='None']
    return cleaned_column"""

"def preprocess_columns(tmp_col):\n    if str(tmp_col) == 'nan':\n        return []\n    evaluated_column = literal_eval(tmp_col)\n    cleaned_column = [item for item in evaluated_column if str(item)!='NOT_MAPPED' and str(item)!='None']\n    return cleaned_column"

In [10]:
"""from ast import literal_eval
columns = [
     'sectors',
     'severity',
     'age',
     'gender',
     'subpillars_1d', 
     'specific_needs_groups',
     'subpillars_2d', 
    ]
def flatten(t):
    return [item for sublist in t for item in sublist]
for column in columns:
    train_val_df[column] = train_val_df[column].apply(preprocess_columns)
"""

"from ast import literal_eval\ncolumns = [\n     'sectors',\n     'severity',\n     'age',\n     'gender',\n     'subpillars_1d', \n     'specific_needs_groups',\n     'subpillars_2d', \n    ]\ndef flatten(t):\n    return [item for sublist in t for item in sublist]\nfor column in columns:\n    train_val_df[column] = train_val_df[column].apply(preprocess_columns)\n"

In [11]:
"""augmented_data = pd.merge(
    right=train_val_df.drop(columns=[
        'excerpt', 'translation_en', 'translation_fr', 'translation_es']
                           ),
    left=augmented_data[['entry_id', 'excerpt']],
    on='entry_id',
    how='right'
)"""

"augmented_data = pd.merge(\n    right=train_val_df.drop(columns=[\n        'excerpt', 'translation_en', 'translation_fr', 'translation_es']\n                           ),\n    left=augmented_data[['entry_id', 'excerpt']],\n    on='entry_id',\n    how='right'\n)"

In [12]:
"""tot_df = pd.concat([train_val_df, augmented_data])"""

'tot_df = pd.concat([train_val_df, augmented_data])'

In [13]:
"""columns = [
     'sectors',
     'severity',
     'age',
     'gender',
     'subpillars_1d', 
     'specific_needs_groups',
     'subpillars_2d', 
    ]
def flatten(t):
    return [item for sublist in t for item in sublist]
tot_df['column_present'] = tot_df.apply(
    lambda x: [column for column in columns if len(x[column])>2], axis=1
               )"""

"columns = [\n     'sectors',\n     'severity',\n     'age',\n     'gender',\n     'subpillars_1d', \n     'specific_needs_groups',\n     'subpillars_2d', \n    ]\ndef flatten(t):\n    return [item for sublist in t for item in sublist]\ntot_df['column_present'] = tot_df.apply(\n    lambda x: [column for column in columns if len(x[column])>2], axis=1\n               )"

In [14]:
"""tot_df.to_csv(os.path.join(DATA_PATH, 'train_val_df.csv'))"""

"tot_df.to_csv(os.path.join(DATA_PATH, 'train_val_df.csv'))"

In [15]:
"""augmented_data = pd.merge(
    right=original_df.drop(columns=['excerpt']),
    left=augmented_data[['entry_id', 'excerpt']],
    on='entry_id',
    how='right'
)"""

"augmented_data = pd.merge(\n    right=original_df.drop(columns=['excerpt']),\n    left=augmented_data[['entry_id', 'excerpt']],\n    on='entry_id',\n    how='right'\n)"

In [16]:
"""tot_df = pd.concat([original_df, augmented_data])"""

'tot_df = pd.concat([original_df, augmented_data])'

In [17]:
"""from ast import literal_eval"""

'from ast import literal_eval'

In [18]:
"""tot_df['pillars_1d'] = tot_df.subpillars_1d.apply(
    lambda item: [x.split('->')[0] for x in literal_eval(item)]
)
tot_df['pillars_2d'] = tot_df.subpillars_2d.apply(
    lambda item: [x.split('->')[0] for x in literal_eval(item)]
)"""

"tot_df['pillars_1d'] = tot_df.subpillars_1d.apply(\n    lambda item: [x.split('->')[0] for x in literal_eval(item)]\n)\ntot_df['pillars_2d'] = tot_df.subpillars_2d.apply(\n    lambda item: [x.split('->')[0] for x in literal_eval(item)]\n)"

In [19]:
columns = ['excerpt', 'entry_id',
           'column_present',
         'sectors',
         'severity',
         'age',
         'gender',
         'subpillars_1d', 
         'specific_needs_groups',
         'subpillars_2d', 
        ]

tot_df = tot_df[columns]

## Sagemaker Prep

### Session

Configure SageMaker

In [20]:
sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
role = SAGEMAKER_ROLE
role_arn = SAGEMAKER_ROLE_ARN
tracking_uri = MLFLOW_SERVER

### Bucket upload

You need to upload data to an S3 bucket. 




In [21]:
MLFLOW_SERVER

'http://mlflow-deep-387470f3-1883319727.us-east-1.elb.amazonaws.com/'

In [22]:
sample = False  # To make the computations faster, sample = True.

if sample:
    tot_df = tot_df.sample(n=20_000)
    
job_name = f"pytorch-{formatted_time()}-all-models"  # change it as you prefer
input_path = DEV_BUCKET / 'training' / 'input_data' / job_name  # Do not change this

train_path = str(input_path / 'train.pickle')
val_path = str(input_path / 'val.pickle')


tot_df.to_pickle(train_path, protocol=4)  # protocol 4 is necessary, since SageMaker uses python 3.6
test_df.to_pickle(val_path, protocol=4)

### Estimator Definition

In [23]:
# GPU instances

instances = [
    'ml.p2.xlarge',
    'ml.p3.2xlarge'
]

The hyperparameters are passed as command line arguments to the training script. 

You can add/change them as you like. It's important to keep the `tracking_uri` and the `experiment_name` which are used by MLFlow.

The class `PyTorch` is part of the `SageMaker` python API. The parameters are important and you should probably not change most of them. The ones you may want to change are:

- `instance_type`, specify the instance you want
- `source_dir`, specify your script directory. Try to use global variable as much as possible

In [24]:
from sagemaker.pytorch import PyTorch

instance_type='ml.p3.2xlarge'

hyperparameters={
    'tracking_uri': MLFLOW_SERVER,
    'experiment_name': "pl-trials",
    'max_len': 128,
    'epochs': 5,
    'model_name': 'microsoft/xtremedistil-l6-h256-uncased',
    'tokenizer_name': 'microsoft/xtremedistil-l6-h256-uncased',
    'dropout_rate': 0.4,
    'output_length': 256,
    'training_names':','.join(columns[2:]),
    "instance_type": instance_type,
    'beta_f1': 0.8
}

estimator = PyTorch(
    entry_point='train_mlflow.py',
    source_dir=str('../../../scripts/training/selim/multiclass-lightning'),
    output_path=str(DEV_BUCKET/'models/'),
    code_location=str(input_path),
    instance_type=instance_type,
    instance_count=1,
    role=role,
    framework_version="1.8",
    py_version="py36",
    hyperparameters = hyperparameters,
    job_name=job_name,
#     train_instance_count=2,
#     train_instance_type="ml.c4.xlarge",
)

In [25]:
fit_arguments = {
    'train': str(input_path),
    'test': str(input_path)
}

In [26]:
# Fit the estimator

estimator.fit(fit_arguments, job_name=job_name)

2021-11-06 14:28:32 Starting - Starting the training job...
2021-11-06 14:28:34 Starting - Launching requested ML instancesProfilerReport-1636208909: InProgress
......
2021-11-06 14:30:05 Starting - Preparing the instances for training......
2021-11-06 14:31:06 Downloading - Downloading input data...
2021-11-06 14:31:46 Training - Downloading the training image.....................
2021-11-06 14:35:27 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-11-06 14:35:18,464 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-11-06 14:35:18,486 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-11-06 14:35:18,492 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-11-06 14:35:18,885 sagemaker-training-toolkit INFO     Installing dependencies fro

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Created wheel for sagemaker: filename=sagemaker-2.49.1-py2.py3-none-any.whl size=591916 sha256=d7cef1aa45c623ca8ddaed8fff742d807a8ea40885fab958a37850a98ed805b7
  Stored in directory: /root/.cache/pip/wheels/4c/af/ea/8ff5943a87155df5b184e54474fbf2b59b75e5c172854643c6
  Created wheel for aiobotocore: filename=aiobotocore-2.0.0-py3-none-any.whl size=50257 sha256=601a9da540ba6e6b9df00287047ebada57db3327e8780c5153a0324b27b17496
  Stored in directory: /root/.cache/pip/wheels/b5/38/68/5d90168a8a21f76350f04a302017488a99ff5ebeac93a721a5
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158155 sha256=97ffb3d7a90a337d0602b2d72e3c2da3f11ae30b4f95cd41fd9c1a250c39c31e
  Stored in directory: /root/.cache/pip/wheels/e9/7b/aa/e18c983d8236b141f85838ba0f8e4e4ae9bcf7f1e00ff726ec
  Created wheel for databricks-cli: filename=databricks_cli-0.16.2-py3-none-any.whl size=106811 sha256=9aa381071fd24f0bcb553cf81cf65e515dca2cc3a36fbb3a7fffaa44addb825c
  Stored in directory: /root/.ca

[2021-11-06 14:37:24.827 algo-1:79 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-11-06 14:37:24.864 algo-1:79 INFO profiler_config_parser.py:102] User has disabled profiler.
[2021-11-06 14:37:24.865 algo-1:79 INFO json_config.py:91] Creating hook from json_config at /opt/ml/input/config/debughookconfig.json.
[2021-11-06 14:37:24.865 algo-1:79 INFO hook.py:201] tensorboard_dir has not been set for the hook. SMDebug will not be exporting tensorboard summaries.
[2021-11-06 14:37:24.866 algo-1:79 INFO hook.py:255] Saving to /opt/ml/output/tensors
[2021-11-06 14:37:24.866 algo-1:79 INFO state_store.py:77] The checkpoint config file /opt/ml/input/config/checkpointconfig.json does not exist.
[2021-11-06 14:37:25.135 algo-1:79 INFO hook.py:594] name:model.l0.embeddings.word_embeddings.weight count_params:7813632
[2021-11-06 14:37:25.136 algo-1:79 INFO hook.py:594] name:model.l0.embeddings.position_embeddings.weight count_params:131072
[2021-11-06 14:37:25.136 algo-1:79 INFO hook.

#015Validation sanity check: 0it [00:00, ?it/s]#015Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]#015                                                              #015#015Training: 99it [00:00, ?it/s]#015Training:   0%|          | 0/12097 [00:00<?, ?it/s]#015Epoch 0:   0%|          | 0/12097 [00:00<?, ?it/s] #015Epoch 0:   0%|          | 30/12097 [00:03<23:25,  8.58it/s]#015Epoch 0:   0%|          | 30/12097 [00:03<23:25,  8.58it/s, loss=1.27, v_num=0, val_loss_epoch=0.667, train_loss=1.050]#015Epoch 0:   0%|          | 60/12097 [00:06<22:42,  8.84it/s, loss=1.27, v_num=0, val_loss_epoch=0.667, train_loss=1.050]#015Epoch 0:   0%|          | 60/12097 [00:06<22:42,  8.83it/s, loss=1.11, v_num=0, val_loss_epoch=0.667, train_loss=1.060]#015Epoch 0:   1%|          | 90/12097 [00:09<22:11,  9.02it/s, loss=1.11, v_num=0, val_loss_epoch=0.667, train_loss=1.060]#015Epoch 0:   1%|          | 90/12097 [00:09<22:11,  9.02it/s, loss=1.04, v_num=0, val_loss_epoch=0.667, train_loss=0.

...

<02:31,  9.15it/s, loss=1.49, v_num=0, val_loss_epoch=0.262, train_loss=1.300, val_loss_step=0.250]#015Epoch 2:  82%|████████▏ | 6510/7892 [11:51<02:31,  9.15it/s, loss=2.37, v_num=0, val_loss_epoch=0.262, train_loss=4.000, val_loss_step=0.250]#015Epoch 2:  83%|████████▎ | 6540/7892 [11:55<02:27,  9.15it/s, loss=2.37, v_num=0, val_loss_epoch=0.262, train_loss=4.000, val_loss_step=0.250]#015Epoch 2:  83%|████████▎ | 6540/7892 [11:55<02:27,  9.15it/s, loss=2.07, v_num=0, val_loss_epoch=0.262, train_loss=1.190, val_loss_step=0.250]#015Epoch 2:  83%|████████▎ | 6570/7892 [11:58<02:24,  9.15it/s, loss=2.07, v_num=0, val_loss_epoch=0.262, train_loss=1.190, val_loss_step=0.250]#015Epoch 2:  83%|████████▎ | 6570/7892 [11:58<02:24,  9.15it/s, loss=2.6, v_num=0, val_loss_epoch=0.262, train_loss=4.630, val_loss_step=0.250] #015Epoch 2:  84%|████████▎ | 6600/7892 [12:01<02:21,  9.15it/s, loss=2.6, v_num=0, val_loss_epoch=0.262, train_loss=4.630, val_loss_step=0.250]#015Epoch 2:  84%|████████▎ | 66

#015Validating: 100%|██████████| 446/446 [00:25<00:00, 17.58it/s]#033[A#015Epoch 2: 100%|██████████| 7892/7892 [14:00<00:00,  9.39it/s, loss=1.72, v_num=0, val_loss_epoch=0.262, train_loss=1.860, val_loss_step=0.249]

2021-11-06 19:40:20 Uploading - Uploading generated training model#015                                                             #033[A#015Epoch 2: 100%|██████████| 7892/7892 [14:17<00:00,  9.20it/s, loss=1.72, v_num=0, val_loss_epoch=0.262, train_loss=1.860, val_loss_step=0.249]#015Epoch 4: 100%|██████████| 666/666 [1:02:00<00:00,  5.59s/it, loss=0.863, v_num=0, val_loss_epoch=0.220, train_loss=1.270, val_loss_step=0.0955]
#015Epoch 2: 100%|██████████| 7892/7892 [22:31<00:00,  5.84it/s, loss=1.72, v_num=0, val_loss_epoch=0.262, train_loss=1.860, val_loss_step=0.249]
2021-11-06 14:36:35.560990: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
INFO:root:reading, preprocessing data
2021/11/06 14:36:38 WA

#015Finding best initial lr: 100%|██████████| 100/100 [00:10<00:00,  9.34it/s]#033[AFIT Profiler Report

Action                             #011|  Mean duration (s)#011|Num calls      #011|  Total time (s) #011|  Percentage %   #011|
--------------------------------------------------------------------------------------------------------------------------------------
Total                              #011|  -              #011|_              #011|  12.522         #011|  100 %          #011|
--------------------------------------------------------------------------------------------------------------------------------------
run_training_epoch                 #011|  11.208         #011|1              #011|  11.208         #011|  89.505         #011|
run_training_batch                 #011|  0.10535        #011|100            #011|  10.535         #011|  84.131         #011|
optimizer_step_and_closure_0       #011|  0.10488        #011|100            #011|  10.488         #011|  83.758   

#015Finding best initial lr: 100%|██████████| 100/100 [1:55:08<00:00, 69.09s/it]




2021-11-06 19:44:21 Completed - Training job completed
Training seconds: 18783
Billable seconds: 18783
